In [1]:
import math
import random
import numpy as np
def criar_rotas_aleatorias_estacao(evrp_data, num_rotas_min, estacao=True):
    '''
    Estrutura da rota: Sempre começa e termina com 1 (depósito), podendo ter outros 1s internos.
    - Clientes (2 até DIMENSION): Aparecem exatamente uma vez.
    - Estações (STATIONS_COORD_SECTION): Opcionais, podem aparecer zero ou múltiplas vezes.
    '''
    dimension = evrp_data['DIMENSION']
    clientes = list(range(2, dimension + 1))  # Todos os clientes (obrigatórios)
    estacoes = evrp_data['STATIONS_COORD_SECTION'] if estacao else []
    
    # Embaralha clientes (garante visita única)
    random.shuffle(clientes)
    
    # Número total de rotas (pelo menos num_rotas_min)
    num_rotas = random.randint(num_rotas_min, num_rotas_min + 3)
    
    # Divide os clientes em 'num_rotas' grupos (sem estações ainda)
    rotas = []
    for i in range(num_rotas):
        elementos_rota = clientes[i::num_rotas]
        rotas.append(elementos_rota)
    
    # Adiciona estações aleatoriamente (opcional)
    if estacao and estacoes:
        for i in range(len(rotas)):
            # Decide quantas estações adicionar nesta rota (pode ser zero)
            num_estacoes = random.randint(0, 3)  # Máximo 3 estações por rota (ajuste conforme necessário)
            posicoes = sorted(random.sample(range(len(rotas[i]) + 1), num_estacoes))  # Posições aleatórias para inserção
            
            # Insere estações nas posições escolhidas
            for j, pos in enumerate(posicoes):
                estacao_escolhida = random.choice(estacoes)
                rotas[i].insert(pos + j, estacao_escolhida)  # +j para compensar inserções anteriores
    
    # Constrói o array final intercalando com 1s (depósito)
    rota_final = [1]
    for rota in rotas:
        rota_final.extend(rota)
        rota_final.append(1)
    
    # Remove 1s consecutivos (rotas vazias)
    i = 1
    while i < len(rota_final) - 1:
        if rota_final[i] == 1 and rota_final[i+1] == 1:
            rota_final.pop(i)
        else:
            i += 1
    
    return np.array(rota_final)

In [2]:
def avaliacao_restricoes(populacao, data, penalidade = 600):
    """
    Calcula fitness considerando distância + penalidades fictícias.
    - todos os VEs começam (carregados e com a bateria cheia) e terminam no depósito;
    - há uma quantidade mínima de rotas/veículos (V);
    - para cada rota de VE, a demanda total de clientes não excede a capacidade máxima de carga (C) do VE;
    - para cada rota de VE, o consumo total de energia não excede o nível máximo de carga da bateria (Q) do VE;
    - os VEs sempre saem do posto de recarga com a bateria totalmente carregada (observe que o depósito também é considerado um posto de recarga); e
    - os postos de recarga (incluindo o depósito) podem ser visitados várias vezes por qualquer VE.
    """
    fitness = {}

    for rota in populacao:
        rota_tuple = tuple(int(node) for node in rota)
        rotas_individuais = [list(rota_tuple[i:j+1]) for i, j in zip([idx for idx, x in enumerate(rota_tuple) if x == 1][:-1], [idx for idx, x in enumerate(rota_tuple) if x == 1][1:])] #Divide as rotas por VE
        
        distancia_total = 0.0
        for rot in rotas_individuais:
            distancia_parcial = 0.0
            capacidade = data['CAPACITY']
            consumo = data['ENERGY_CAPACITY']

            for i in range(len(rot) - 1):
                x1, y1 = data['NODE_COORD_SECTION'][rot[i]]
                x2, y2 = data['NODE_COORD_SECTION'][rot[i + 1]]
                dist = math.sqrt((x2 - x1)**2 + (y2 - y1)**2)

                distancia_parcial += dist
                #1.para cada rota de VE, a demanda total de clientes não excede a capacidade máxima de carga (C) do VE;
                if rot[i + 1] not in data['STATIONS_COORD_SECTION']: #Se meu proximo não for estação (deposito tem valor 0 como padrão)
                    capacidade -= data['DEMAND_SECTION'][rot[i + 1]]

                #2. para cada rota de VE, o consumo total de energia não excede o nível máximo de carga da bateria (Q) do VE;
                consumo -= dist * data['ENERGY_CONSUMPTION']
                if consumo >= 0 and (rot[i + 1] in data['STATIONS_COORD_SECTION'] or rot[i + 1] == 1): #Se meu proximo destino e estação/deposito e eu ainda tenho energia
                    consumo = data['ENERGY_CAPACITY'] #Recarrego tudo
            
            if capacidade < 0:
                print(f"Capacidade ultrapassada na rota: {rot} em {data['CAPACITY']-capacidade}")
                distancia_total += penalidade
            if consumo < 0: 
                print(f"Consumo ultrapassada na rota: {rot} em {data['ENERGY_CAPACITY']-consumo}")
                distancia_total += penalidade
            distancia_total += distancia_parcial

    
        fitness[rota_tuple] = 1 / (distancia_total + 1e-6)  # +1e-6 evita divisão por zero
    return fitness

In [3]:
import numpy as np
from GA.operators.export import *
from GA.utils.export import *

class EVRP_GA:
    def __init__(self, filename, param_ga, config, estrat = "", binario=True, restricoes=True, id = None):
        self.evrp_data = read_evrp_file(filename)
        self.param_problema = parametros_problema(self.evrp_data, binario, restricoes)
        self.param_ga = param_ga
        self.config = config
        self.id = id
        self.nameEstrat = estrat
        self.best_run = []
        self.best_dist = float('inf')
        self.population = []
        self.fitness = {}
        self.pais = []
        self.filhos = []
        self.new_pop = []
        self.filename = filename
        # Contadores e histórico
        self.n_aval = 0  # Contador de avaliações
        self.historico_melhor_rota = []
        self.historico_melhor_distancia = []

        # Configuração dos operadores
        self.evaluation_methods = {
            'distancia': avaliacao_distancia_pura, #
            'restricoes': avaliacao_restricoes,
            'rankeamento': avaliacao_rankeamento 
        }

        self.selection_methods = {
            'roleta': selecao_roleta,
            'torneio': selecao_torneio, #
            'rank': selecao_rank
        }
        
        self.crossover_methods = {
            'one_point': crossover_one_point,
            'two_point': crossover_two_point,
            'uniforme': crossover_uniforme
        }

        self.mutation_methods = {
            'bit_flip': mutacao_bit_flip,
            'swap': mutacao_swap,
            'inversao': mutacao_inversao,
            'scramble': mutacao_scramble,
            '': False
        }

        self.replacement_methods = {
            'completa': substituicao_completa,
            'elitimos': substituicao_elitismo, #
            'steady_state': substituicao_steady_state,
        }

    def initialize(self):
        n_pop = self.param_ga['n_pop']
        self.population = [criar_rotas_aleatorias_estacao(self.evrp_data, self.param_problema['num_rotas_min'], self.param_problema['restricoes']) for _ in range(n_pop)]
        #print(f"Iniciou com: {len(self.population)} rotas")
    
    def evaluate(self):
        """Avalia a população e atualiza o contador de avaliações."""
        metodo = self.evaluation_methods[self.config['evaluation']]
        self.fitness = metodo(self.population, self.evrp_data)
        self.n_aval += len(self.population)  # Incrementa o contador
        #print(f"Avaliou {len(self.fitness)} rotas (Total de avaliações: {self.n_aval})")

    def selection(self):
        metodo = self.selection_methods[self.config['selection']]
        self.pais = metodo(self.population, self.fitness, self.param_ga['n_pais'])
        #print(f"Seleciou {len(self.pais)} pais")

    def crossover(self):
        self.filhos = crossover_completo(self.pais, self.evrp_data, n_filhos = self.param_ga['n_filhos'], num_rotas_min=self.param_problema['num_rotas_min'], tipo_crossover=self.config['crossover'], taxa_crossover=1, estacao=self.param_problema['restricoes'])
        #print(f"Gerou {len(self.filhos)} filhos")

    def mutation(self):
        if self.config['mutation'] == '': return
        self.filhos = aplicar_mutacao(self.filhos, self.evrp_data, self.param_problema['num_rotas_min'], metodo=self.config['mutation'], taxa_mutacao=0.1, estacao=self.param_problema['restricoes'])
        #print(f"Mutação em {len(self.filhos)} filhos")

    def replacement(self):
        fitness_filhos = avaliacao_rankeamento(self.filhos, self.evrp_data)
        self.new_pop = gerar_nova_populacao(self.population, self.filhos, self.fitness, fitness_filhos, metodo=self.config['replacement'], 
                         n_pop=self.param_ga['n_pop'], n_pais=self.param_ga['n_pais'], n_filhos=self.param_ga['n_filhos'], n_elite=5)
        #print(f"Finalizou com: {len(self.new_pop)} rotas")

    def registrar_melhoria_csv(self, onde):
        """Registra uma nova melhoria no CSV"""
        if self.id != None:
            with open(f'results/csvs/melhores_resultados3_{self.filename}_{self.nameEstrat}_{self.id}.csv', 'a', newline='') as csvfile:
                writer = csv.writer(csvfile)
                writer.writerow([
                    self.n_aval,
                    self.best_dist,
                    onde,
                    self.config["crossover"],
                    self.config["mutation"],
                ])

        print(f"Parada atingida: {self.n_aval} avaliações realizadas.")
        return self.best_run
    

    def escolher_metodo(self,probabilidades):
        return random.choices(range(4), weights=probabilidades, k=1)[0]

    def atualizar_pesos(self,sucessos, tentativas):
        base = 0.01  # para evitar pesos zero
        pesos = [s / t + base for s, t in zip(sucessos, tentativas)]
        soma = sum(pesos)
        return [p / soma for p in pesos]

    def run_2(self):
        self.initialize()
        self.evaluate()
        #criar_csv_vazio()
        
        # Contadores e estado para controle de convergência
        contador_estagnacao = 0
        MAX_ESTAGNACAO = 100  # Número de iterações sem melhoria para considerar convergência
        MELHORIA_MINIMA = 1.0  # Melhoria mínima para resetar contador
        
        # Listas de métodos para rotação
        crossover_methods = ['one_point', 'two_point', 'uniforme', 'OX']
        mutation_methods = ['swap', 'inversao', 'scramble', 'insercao']

        vetCross = [0.25] * 4
        vetMut =  [0.25] * 4
        sucessoCross = [1] * 4  # Começa com 1 para evitar divisão por zero
        tentativasCross = [1] * 4

        sucessoMut = [1] * 4
        tentativasMut = [1] * 4

        idx_cross = self.escolher_metodo(vetCross)
        idx_mut = self.escolher_metodo(vetMut)

        
        self.config["crossover"] = crossover_methods[idx_cross]
        self.config["mutation"] = mutation_methods[idx_mut]
        print(f"Métodos: Crossover={self.config['crossover']}, Mutação={self.config['mutation']}")
        # Histórico de melhor distância para detecção de convergência
        last_best_dist = float('inf')
        
        while self.n_aval < self.param_ga['max_aval']:
            # Etapa normal do GA
            best_rota, best_dist = melhor_rota(self.population, self.evrp_data)
            
            # Verifica melhoria
            if last_best_dist - best_dist > MELHORIA_MINIMA:
                contador_estagnacao = 0
                last_best_dist = best_dist
            else:
                contador_estagnacao += 1
            
            # Atualiza melhor global
            if self.best_dist > best_dist:
                print(f"Melhoria na iteração {self.n_aval}: {best_dist:.2f}")
                self.best_dist = best_dist
                self.best_run = best_rota
                self.registrar_melhoria_csv('Populacao')
            
            # Verifica convergência
            if contador_estagnacao >= MAX_ESTAGNACAO:
                print(f"Convergência detectada na iteração {self.n_aval}. Reiniciando população parcialmente...")
                
                # 1. Mantém os melhores indivíduos (elitismo)
                elite_size = int(0.2 * self.param_ga['n_pop'])  # 20% da população
                elite = sorted(self.population, key=lambda x: self.fitness[tuple(x)])[:elite_size]
                
                # 2. Gera nova população aleatória para o restante
                new_random = [criar_rotas_aleatorias(self.evrp_data, 
                            self.param_problema['num_rotas_min'], 
                            self.param_problema['restricoes']) 
                            for _ in range(self.param_ga['n_pop'] - elite_size)]
                
                # 3. Combina elite + novos indivíduos
                self.population = elite + new_random
                self.evaluate()
                
                # 4. Rota os métodos de crossover e mutação
                idx_cross = self.escolher_metodo(vetCross)
                idx_mut = self.escolher_metodo(vetMut)
                self.config["crossover"] = crossover_methods[idx_cross]
                self.config["mutation"] = mutation_methods[idx_mut]
                print(f"Novos métodos: Crossover={self.config['crossover']}, Mutação={self.config['mutation']}")
                
                # 5. Reseta contador de estagnação
                contador_estagnacao = 0
            
            # Processo normal do GA
            self.historico_melhor_rota.append(best_rota)
            self.historico_melhor_distancia.append(best_dist)
            
            self.selection()
            self.crossover()
            
            best_rota_filhos, best_dist_filhos = melhor_rota(self.filhos, self.evrp_data)
            if self.best_dist > best_dist_filhos:
                print(f"Melhoria na iteração {self.n_aval}: {best_dist:.2f} , Crossover")
                self.best_dist = best_dist_filhos
                self.best_run = best_rota_filhos
                sucessoCross[idx_cross] += 1
                self.registrar_melhoria_csv('Crossover')
            tentativasCross[idx_cross] += 1
    
            self.mutation()
            best_rota_filhos, best_dist_filhos = melhor_rota(self.filhos, self.evrp_data)
            if self.best_dist > best_dist_filhos:
                print(f"Melhoria na iteração {self.n_aval}: {best_dist:.2f} , Mutation")
                self.best_dist = best_dist_filhos
                self.best_run = best_rota_filhos
                self.registrar_melhoria_csv('Mutacao')
                sucessoMut[idx_mut] += 1
            tentativasMut[idx_mut] += 1
            self.replacement()
            self.population = self.new_pop
            self.evaluate()
            if self.n_aval % 100 == 0:
                vetCross = self.atualizar_pesos(sucessoCross, tentativasCross)
                vetMut = self.atualizar_pesos(sucessoMut, tentativasMut)
        print(f"Parada atingida: {self.n_aval} avaliações realizadas.")
        return self.best_run



    def mostrar_historico(self):
        """Exibe o histórico de melhores distâncias por geração."""
        import matplotlib.pyplot as plt
        plt.plot(self.historico_melhor_distancia, marker='o')
        plt.xlabel('Geração')
        plt.ylabel('Melhor Distância')
        plt.title('Convergência do Algoritmo')
        plt.grid(True)
        plt.show()
        
        # Exibe detalhes
        for geracao, distancia in enumerate(self.historico_melhor_distancia):
            print(f"Geração {geracao}: {distancia:.2f}")

In [4]:
# 3 das combinações (dist, roleta ou torneio, two_point ou uniforme, swap ou scramble ou inverso, elitismo)
# 20 da melhor

params_ga = {
    'n_pop': 50,                # População maior para mais diversidade
    'n_geracoes': 500,
    'max_aval': 25000*32,
    'taxa_crossover': 1,
    'taxa_mutacao': 0.7,        # Mutação baixa para evitar perturbações excessivas
    'n_pais': 30,
    'n_filhos': 100,
    'n_elite': 10,
    'num_rotas_min': 3,          # Definido pelo problema
    'limite_convergencia': 20    # Parar se não houver melhoria em 20 gerações
}

config = {
    'evaluation': 'distancia',  # Usar penalidades para restrições distancia/restricoes
    'selection': 'torneio',      # Torneio é mais eficiente para evitar convergência prematura
    'tamanho_torneio': 5,
    'crossover': 'two_point',    # Combinação mais equilibrada
    'mutation': 'swap',          # Mutação simples para preservar estrutura
    'replacement': 'elitismo'    # Mantém as melhores soluções
}

In [12]:
random.seed(1)
ga = EVRP_GA('E-n23-k3.evrp', params_ga, config)
ga.evrp_data

5
5


{'COMMENT': 'Modificatification of E-n23-k3 instance. Modified by Mavrovouniotis Menelaou.',
 'OPTIMAL_VALUE': 573.130948,
 'VEHICLES': 3,
 'DIMENSION': 23,
 'STATIONS': 9,
 'CAPACITY': 4500,
 'ENERGY_CAPACITY': 190.0,
 'ENERGY_CONSUMPTION': 1.2,
 'NODE_COORD_SECTION': {1: (266.0, 235.0),
  2: (295.0, 272.0),
  3: (301.0, 258.0),
  4: (309.0, 260.0),
  5: (217.0, 274.0),
  6: (218.0, 278.0),
  7: (282.0, 267.0),
  8: (242.0, 249.0),
  9: (230.0, 262.0),
  10: (249.0, 268.0),
  11: (256.0, 267.0),
  12: (265.0, 257.0),
  13: (267.0, 242.0),
  14: (259.0, 265.0),
  15: (315.0, 233.0),
  16: (329.0, 252.0),
  17: (318.0, 252.0),
  18: (329.0, 224.0),
  19: (267.0, 213.0),
  20: (275.0, 192.0),
  21: (303.0, 201.0),
  22: (208.0, 217.0),
  23: (326.0, 181.0),
  24: (229.0, 198.0),
  25: (229.0, 230.0),
  26: (229.0, 262.0),
  27: (269.0, 198.0),
  28: (269.0, 230.0),
  29: (269.0, 262.0),
  30: (309.0, 198.0),
  31: (309.0, 230.0),
  32: (309.0, 262.0)},
 'DEMAND_SECTION': {1: 0,
  2: 125,

In [195]:
base = [criar_rotas_aleatorias_estacao(evrp_data, 3, True) for _ in range(10)]
#rota_int = tuple(int(node) for node in base)
#rotas_individuais = [list(rota_int[i:j+1]) for i, j in zip([idx for idx, x in enumerate(rota_int) if x == 1][:-1], [idx for idx, x in enumerate(rota_int) if x == 1][1:])]
#rotas_individuais

In [196]:
base

[array([ 1, 10, 11, 32, 13, 27, 22, 32,  1, 27, 14, 32, 19, 25,  9, 17,  1,
        21, 30,  2, 12, 16,  1, 27, 23, 29, 18, 25,  4,  5,  1,  8,  6,  7,
         1,  3, 29, 15, 25, 20,  1]),
 array([ 1,  8, 20, 13, 24, 11, 14, 29,  3,  1,  5, 18, 22, 24, 16,  4, 10,
         1, 23, 21,  6,  7, 32, 12,  1, 15, 17, 19,  9,  2,  1]),
 array([ 1, 13, 30, 19, 22, 29,  5,  1,  3, 20, 15,  2,  1, 16, 32, 23,  8,
        28, 10,  1,  6, 17, 24, 18, 28,  4, 26,  1, 31, 12, 11, 14, 31,  1,
        28,  7, 31,  9, 21, 32,  1]),
 array([ 1, 30, 14, 21, 31,  5, 20, 16, 27, 15,  1, 12, 28, 10,  3, 30, 22,
        23, 31, 11,  1, 13,  6, 28, 18, 30,  9,  7,  1,  8,  2, 17, 27, 19,
         4, 32,  1]),
 array([ 1, 23,  6, 17,  9,  3, 22, 24, 15, 26, 20, 27,  1,  8, 25, 16, 19,
        13, 12, 27,  2, 10,  1, 21,  4,  5, 32,  7, 18, 14, 11,  1]),
 array([ 1, 28, 16, 27, 15,  3,  7, 23,  1, 22, 31,  9,  2, 21, 28,  4, 30,
         1, 11,  5, 29,  6, 12,  1, 31, 20, 13, 14, 10,  1, 18, 17, 19,  8,
      

In [197]:
fitness_1 = avaliacao_distancia_pura(base, evrp_data)
print(fitness_1)

fitness_2 = avaliacao_restricoes(base, evrp_data)
print(fitness_2)

{(1, 10, 11, 32, 13, 27, 22, 32, 1, 27, 14, 32, 19, 25, 9, 17, 1, 21, 30, 2, 12, 16, 1, 27, 23, 29, 18, 25, 4, 5, 1, 8, 6, 7, 1, 3, 29, 15, 25, 20, 1): 0.00044699274602251666, (1, 8, 20, 13, 24, 11, 14, 29, 3, 1, 5, 18, 22, 24, 16, 4, 10, 1, 23, 21, 6, 7, 32, 12, 1, 15, 17, 19, 9, 2, 1): 0.0006159205847824548, (1, 13, 30, 19, 22, 29, 5, 1, 3, 20, 15, 2, 1, 16, 32, 23, 8, 28, 10, 1, 6, 17, 24, 18, 28, 4, 26, 1, 31, 12, 11, 14, 31, 1, 28, 7, 31, 9, 21, 32, 1): 0.0004490748841597187, (1, 30, 14, 21, 31, 5, 20, 16, 27, 15, 1, 12, 28, 10, 3, 30, 22, 23, 31, 11, 1, 13, 6, 28, 18, 30, 9, 7, 1, 8, 2, 17, 27, 19, 4, 32, 1): 0.0004899988557740768, (1, 23, 6, 17, 9, 3, 22, 24, 15, 26, 20, 27, 1, 8, 25, 16, 19, 13, 12, 27, 2, 10, 1, 21, 4, 5, 32, 7, 18, 14, 11, 1): 0.0005186524881036845, (1, 28, 16, 27, 15, 3, 7, 23, 1, 22, 31, 9, 2, 21, 28, 4, 30, 1, 11, 5, 29, 6, 12, 1, 31, 20, 13, 14, 10, 1, 18, 17, 19, 8, 1): 0.0005767552580803154, (1, 22, 25, 9, 28, 23, 10, 1, 12, 20, 13, 18, 1, 7, 16, 3, 24,

In [ ]:
pais = selecao_torneio(base, fitness_2, 2)
#pais
base

In [198]:
import numpy as np
import math

def aplicar_restricao(rota, evrp_data, num_rotas_min=3):
    """
    Transforma uma rota em uma solução válida para o EVRP, aplicando todas as restrições:
    1. Todos os VEs começam e terminam no depósito.
    2. Número mínimo de rotas/veículos.
    3. Capacidade máxima de carga por VE.
    4. Nível máximo de bateria por VE.
    5. Recarga total em estações/depósito.
    6. Visitas únicas a clientes.
    
    Args:
        rota: Array numpy representando a rota (ex: [1, 24, 21, ..., 1]).
        evrp_data: Dicionário com os dados do problema.
        num_rotas_min: Número mínimo de rotas exigido.
    
    Returns:
        Rota válida (array numpy) com todas as restrições aplicadas.
    """
    # --- 0. Prepara a rota ---
    # Se a rota for uma lista contendo um array, pega o array
    if isinstance(rota, list) and len(rota) == 1 and isinstance(rota[0], np.ndarray):
        rota = rota[0]
    
    # --- 1. Pré-processamento: Divide a rota em sub-rotas por veículo ---
    rotas = []
    current_route = [1]  # Todas as rotas começam no depósito (1)
    
    for node in rota[1:]:  # Ignora o primeiro 1 (já adicionado)
        if node == 1 and len(current_route) > 1:  # Encontrou um novo depósito (fim da rota atual)
            current_route.append(1)
            rotas.append(current_route)
            current_route = [1]
        else:
            current_route.append(node)
    
    if len(current_route) > 1:  # Adiciona a última rota se não terminou com 1
        current_route.append(1)
        rotas.append(current_route)
    
    
    # --- 2. Garante o número mínimo de rotas ---
    while len(rotas) < num_rotas_min:
        if rotas:  # Se houver rotas para dividir
            maior_rota_idx = max(range(len(rotas)), key=lambda i: len(rotas[i]))
            maior_rota = rotas[maior_rota_idx]
            meio = len(maior_rota) // 2
            nova_rota1 = maior_rota[:meio] + [1]
            nova_rota2 = [1] + maior_rota[meio:]
            rotas[maior_rota_idx] = nova_rota1
            rotas.insert(maior_rota_idx + 1, nova_rota2)
        else:
            rotas = [[1, 1] for _ in range(num_rotas_min)]
            break
    
    # --- 3. Remove clientes duplicados (visita única) ---
    clientes_visitados = set()
    clientes_validos = set(range(2, evrp_data['DIMENSION'] + 1))
    
    for i in range(len(rotas)):
        rota = rotas[i]
        nova_rota = [1]
        for node in rota[1:-1]:  # Ignora os depósitos inicial/final
            if node in clientes_validos:
                if node not in clientes_visitados:
                    nova_rota.append(node)
                    clientes_visitados.add(node)
            else:  # Estação de recarga (pode ser repetida)
                nova_rota.append(node)
        nova_rota.append(1)
        rotas[i] = nova_rota
    
    # --- 4. Reconstrói a rota completa ---
    rota_final = []
    for rota in rotas:
        rota_final.extend(rota)

    # 5. Remove 1s consecutivos (rotas vazias)
    i = 1
    while i < len(rota_final) - 1:
        if rota_final[i] == 1 and rota_final[i+1] == 1:
            rota_final.pop(i)
        else:
            i += 1

    # --- 6. Aplica restrições de capacidade e bateria ---
    carga = [evrp_data['CAPACITY']]
    carga_atual = carga[-1]
    bateria = [evrp_data['ENERGY_CAPACITY']]
    atual = 1

    print(f"Antes da bomba: {tuple(int(node) for node in rota_final)}")
    while atual < len(rota_final):
        #Define o atual como destino, e verifica se ele pode entrar
        # print(f"Sit: {tuple(int(node) for node in rota_final[:atual])}")
        origem = rota_final[atual-1]
        destino = rota_final[atual]
        x1, y1 = evrp_data['NODE_COORD_SECTION'][origem]
        x2, y2 = evrp_data['NODE_COORD_SECTION'][destino]
        distancia = math.sqrt((x2 - x1)**2 + (y2 - y1)**2)
        consumo = evrp_data['ENERGY_CONSUMPTION'] * distancia
        if destino not in evrp_data['STATIONS_COORD_SECTION']:
            carga_atual = carga[-1] - evrp_data['DEMAND_SECTION'][destino]
        bateria_atual = bateria[-1] - consumo

        #Se o destino não quebra restrições, passamos para o proximo, armazenando os valores de bateria e carga
        # print(f"Atual: {atual}, Destino: {rota_final[atual]}")
        # print(f"CAntes: {carga[-1]}, CDepois {carga_atual}")
        # print(f"BAntes: {bateria[-1]}, BDepois {bateria_atual}")
        # print(f"Tamanhos: A{atual}, C{len(carga)}, B{len(bateria)}")
        # print()
        if bateria_atual > 0 and carga_atual > 0:
            if rota_final[atual] == 1:
                bateria.append(evrp_data['ENERGY_CAPACITY'])
                carga.append(evrp_data['CAPACITY'])
            elif rota_final[atual] in evrp_data['STATIONS_COORD_SECTION']:
                bateria.append(evrp_data['ENERGY_CAPACITY'])
                carga.append(carga[-1])
            else:
                bateria.append(bateria_atual)
                carga.append(carga_atual)
            atual += 1

        elif bateria_atual < 0: #Se não tiver bateria, vai retrocedendo ate conseguir colocar a estação mais proxima
            estacao_encontrada = None
            while atual >= 0 and estacao_encontrada is None:
                # Ponto atual para procurar estação mais próxima
                ponto_atual = rota_final[atual-1]
                xp, yp = evrp_data['NODE_COORD_SECTION'][ponto_atual]
                
                # Encontra estação mais próxima deste ponto, e define o consumo ate ela
                estacoes = evrp_data['STATIONS_COORD_SECTION']
                if estacoes:
                    estacao_proxima = min(
                        estacoes,
                        key=lambda e: math.sqrt(
                            (xp - evrp_data['NODE_COORD_SECTION'][e][0])**2 +
                            (yp - evrp_data['NODE_COORD_SECTION'][e][1])**2
                        )
                    )
                    xe, ye = evrp_data['NODE_COORD_SECTION'][estacao_proxima]
                    distancia_e = math.sqrt((xe - xp)**2 + (ye - yp)**2)
                    consumo_e = evrp_data['ENERGY_CONSUMPTION'] * distancia_e

                    #Se não tem energia, retrocede um cliente e tenta denovo, caso contrario, achamos a estação
                    #print(f"BatNeg: {atual-1}, size: {len(bateria)}, ponto {ponto_atual}, est: {estacao_proxima}, last: {bateria[-1]}, gasto: {consumo_e}")
                    if bateria[atual-1] - consumo_e < 0:
                        atual -= 1
                    else:
                        estacao_encontrada = estacao_proxima
            if estacao_encontrada is None:
                print("Fudeu de maneiras que eu não sei explicar") #Como caralhos que uma rota que esta na porra do deposito não consegue ir para nenhum lugar sem gastar energia e loucura
            else: 
                bateria = bateria[:atual]
                carga = carga[:atual]

                rota_final.insert(atual, estacao_encontrada)
                bateria.append(evrp_data['ENERGY_CAPACITY'])
                carga.append(carga[-1])

                atual += 1
        else: #Carga atual negativada
            end_route = False
            x2, y2 = evrp_data['NODE_COORD_SECTION'][1]
            #print(f"{end_route} and {rota_final[atual-1]}")
            while not end_route and rota_final[atual-1]!=1:
                x1, y1 = evrp_data['NODE_COORD_SECTION'][rota_final[atual-1]]
                distancia = math.sqrt((x2 - x1)**2 + (y2 - y1)**2)
                consumo = evrp_data['ENERGY_CONSUMPTION'] * distancia
                bateria_atual = bateria[atual-1] - consumo
                if bateria_atual > 0:
                    end_route = True
                    bateria = bateria[:atual]
                    carga = carga[:atual]

                    rota_final.insert(atual, 1)
                    bateria.append(evrp_data['ENERGY_CAPACITY'])
                    carga.append(evrp_data['CAPACITY'])
                    atual +=1
                else:
                    atual -= 1
            if rota_final[atual] == 1:
                print("Vou me matar as 23:45")
    print(f"Depois da bomba: {tuple(int(node) for node in rota_final)}")
    return np.array(rota_final)

In [199]:
#base = criar_rotas_aleatorias_estacao(evrp_data, 3, True)
#rotas_boa = aplicar_restricao(base, evrp_data, num_rotas_min=3)

base = [criar_rotas_aleatorias_estacao(evrp_data, 3, True) for _ in range(1000)]
rotas_boa = [aplicar_restricao(rotas, evrp_data, num_rotas_min=3) for rotas in base]

Antes da bomba: (1, 17, 14, 8, 25, 4, 11, 21, 26, 1, 16, 2, 28, 22, 3, 18, 23, 27, 1, 6, 20, 15, 32, 10, 9, 1, 12, 13, 30, 5, 31, 19, 7, 1)
Depois da bomba: (1, 17, 14, 8, 26, 25, 4, 1, 11, 1, 21, 26, 1, 16, 2, 28, 22, 25, 3, 18, 31, 23, 27, 1, 6, 26, 20, 15, 31, 32, 10, 9, 1, 12, 13, 30, 5, 26, 31, 19, 7, 1)
Antes da bomba: (1, 31, 18, 13, 6, 5, 10, 14, 9, 17, 1, 11, 25, 2, 3, 21, 15, 7, 25, 12, 1, 19, 8, 16, 23, 4, 22, 20, 1)
Depois da bomba: (1, 31, 18, 13, 28, 6, 5, 10, 14, 9, 26, 17, 1, 11, 25, 2, 3, 1, 21, 15, 7, 29, 25, 12, 1, 19, 8, 26, 16, 32, 23, 30, 4, 32, 22, 25, 20, 1)
Antes da bomba: (1, 23, 12, 15, 18, 28, 1, 10, 17, 30, 2, 27, 7, 1, 6, 28, 5, 24, 4, 19, 1, 25, 13, 8, 3, 14, 1, 16, 29, 22, 20, 1, 11, 25, 21, 9, 1)
Depois da bomba: (1, 23, 30, 12, 15, 31, 18, 28, 1, 10, 17, 32, 30, 2, 27, 7, 1, 6, 28, 5, 24, 4, 32, 19, 1, 25, 13, 8, 3, 32, 14, 1, 16, 29, 22, 20, 27, 1, 11, 25, 1, 21, 9, 26, 1)
Antes da bomba: (1, 8, 22, 32, 21, 7, 25, 1, 31, 3, 5, 31, 16, 31, 4, 1, 23, 9,

In [202]:
fitness_1 = avaliacao_distancia_pura(rotas_boa, evrp_data)
print(fitness_1)

fitness_2 = avaliacao_restricoes(rotas_boa, evrp_data)
print(fitness_2)

{(1, 17, 14, 8, 26, 25, 4, 1, 11, 1, 21, 26, 1, 16, 2, 28, 22, 25, 3, 18, 31, 23, 27, 1, 6, 26, 20, 15, 31, 32, 10, 9, 1, 12, 13, 30, 5, 26, 31, 19, 7, 1): 0.0005098753625929897, (1, 31, 18, 13, 28, 6, 5, 10, 14, 9, 26, 17, 1, 11, 25, 2, 3, 1, 21, 15, 7, 29, 25, 12, 1, 19, 8, 26, 16, 32, 23, 30, 4, 32, 22, 25, 20, 1): 0.000654492699814638, (1, 23, 30, 12, 15, 31, 18, 28, 1, 10, 17, 32, 30, 2, 27, 7, 1, 6, 28, 5, 24, 4, 32, 19, 1, 25, 13, 8, 3, 32, 14, 1, 16, 29, 22, 20, 27, 1, 11, 25, 1, 21, 9, 26, 1): 0.00047113744833930914, (1, 8, 22, 25, 32, 21, 7, 29, 25, 1, 31, 3, 5, 26, 31, 16, 31, 4, 1, 23, 30, 9, 10, 14, 29, 1, 13, 28, 11, 25, 1, 6, 12, 1, 17, 18, 31, 24, 15, 1, 26, 20, 27, 2, 32, 19, 1): 0.0004934476986686042, (1, 5, 26, 17, 32, 24, 1, 11, 4, 32, 9, 28, 1, 20, 7, 29, 16, 15, 13, 1, 21, 3, 2, 32, 10, 1, 8, 26, 23, 30, 19, 22, 25, 1, 18, 12, 14, 29, 6, 1): 0.000585150626327813, (1, 23, 30, 3, 32, 6, 26, 30, 7, 9, 26, 4, 32, 1, 15, 21, 19, 27, 25, 12, 25, 16, 32, 14, 1, 32, 8, 11

In [ ]:
def reparar_filho(filho, pai, evrp_data, num_rotas_min=3, estacao=False):
    """
    Corrige uma rota filho com base no pai para atender às restrições do EVRP.
    
    Args:
        filho: Rota filho a ser reparada (array numpy).
        pai: Rota pai usada como referência (array numpy).
        evrp_data: Dicionário com dados do problema.
        num_rotas_min: Número mínimo de rotas exigido.
        estacao: Se True, inclui estações de recarga como nós válidos.
    
    Returns:
        Rota filho reparada (array numpy).
    """
    # Converte para lista para facilitar manipulação
    filho_list = filho.tolist()
    pai_list = pai.tolist()
    
    # 1. Garante que começa e termina com 1
    if filho_list[0] != 1:
        filho_list.insert(0, 1)
    if filho_list[-1] != 1:
        filho_list.append(1)
    
    # 2. Remove 1s consecutivos (rotas vazias)
    i = 1
    while i < len(filho_list) - 1:
        if filho_list[i] == 1 and filho_list[i+1] == 1:
            filho_list.pop(i)
        else:
            i += 1
    
    # 3. Identifica elementos válidos e clientes faltantes
    dimension = evrp_data['DIMENSION']
    
    clientes_validos = set(range(2, dimension + 1))
    estacoes_validas = set(evrp_data['STATIONS_COORD_SECTION']) if estacao else set()
    elementos_validos = clientes_validos.union(estacoes_validas)
    
    # 4. Remove clientes inválidos (0, IDs maiores que DIMENSION)
    visitas_presentes = []
    for node in filho_list[1:-1]:  # Ignora o primeiro e último 1
        if node in elementos_validos:
            visitas_presentes.append(node)
    
    # 5. Remove duplicatas mantendo a ordem de primeira ocorrência
    clientes_unicos = []
    seen = set()
    for node in visitas_presentes:
        if node in clientes_validos and node not in seen:
            seen.add(node)
            clientes_unicos.append(node)
    
    # 6. Completa com clientes faltantes do pai (se necessário)
        clientes_faltantes = list(clientes_validos - set(clientes_unicos))
        # Adiciona clientes faltantes do pai que não estão no filho
        for node in pai_list[1:-1]:
            if node in clientes_faltantes and node not in clientes_unicos:
                clientes_unicos.append(node)
                clientes_faltantes.remove(node)
    
    # 7. Reconstroi a rota com os clientes válidos
    # Divide em rotas baseado nos 1s do pai (estrutura herdada)
    rotas_pai = []
    current_route = []
    for node in pai_list[1:-1]:  # Ignora o primeiro e último 1
        if node == 1:
            if current_route:
                rotas_pai.append(current_route)
                current_route = []
        else:
            current_route.append(node)
    if current_route:
        rotas_pai.append(current_route)
    
    # Distribui os clientes válidos nas rotas do pai
    filho_reparado = [1]
    clientes_alocados = 0
    
    for rota_pai in rotas_pai:
        if not clientes_unicos:
            break
            
        # Pega os clientes necessários para esta rota
        num_clientes_rota = min(len(rota_pai), len(clientes_unicos) - clientes_alocados)
        rota_filho = clientes_unicos[clientes_alocados:clientes_alocados + num_clientes_rota]
        filho_reparado.extend(rota_filho)
        filho_reparado.append(1)
        clientes_alocados += num_clientes_rota
    
    # Adiciona clientes restantes (se houver) em novas rotas
    while clientes_alocados < len(clientes_unicos):
        num_restante = len(clientes_unicos) - clientes_alocados
        rota_filho = clientes_unicos[clientes_alocados:clientes_alocados + min(num_restante, 10)]  # Máx 10 clientes por rota
        filho_reparado.extend(rota_filho)
        filho_reparado.append(1)
        clientes_alocados += len(rota_filho)
    
    # 8. Garante número mínimo de rotas
    num_rotas = filho_reparado.count(1) - 1

    if num_rotas < num_rotas_min:
        # Divide a maior rota em duas para aumentar o número de rotas
        rotas = []
        current = []
        for node in filho_reparado:
            if node == 1:
                if current:
                    rotas.append(current)
                    current = []
            else:
                current.append(node)
        
        while len(rotas) < num_rotas_min and len(rotas) > 0:
            # Encontra a rota mais longa
            maior_rota_idx = max(range(len(rotas)), key=lambda i: len(rotas[i]))
            maior_rota = rotas[maior_rota_idx]
            
            if len(maior_rota) >= 2:
                # Divide no meio
                meio = len(maior_rota) // 2
                nova_rota1 = maior_rota[:meio]
                nova_rota2 = maior_rota[meio:]
                
                # Substitui a rota original pelas duas novas
                rotas.pop(maior_rota_idx)
                rotas.insert(maior_rota_idx, nova_rota1)
                rotas.insert(maior_rota_idx + 1, nova_rota2)
        
        # Reconstrói o filho
        filho_reparado = [1]
        for rota in rotas:
            filho_reparado.extend(rota)
            filho_reparado.append(1)

    filho_reparado = aplicar_restricao(filho_reparado, evrp_data, num_rotas_min=3)

    return np.array(filho_reparado)

In [ ]:
filhos = crossover_completo(base, evrp_data, 2, num_rotas_min=3, tipo_crossover='one_point', taxa_crossover=0.8, estacao=True)
filhos

In [ ]:
rota = ga.run_2()
print(type(rota))
print(rota)

In [ ]:
print(calcular_distancia_total(ga.evrp_data, rota))
plot_single_route_with_trips(ga.evrp_data, rota)
plot_evrp_instance(ga.evrp_data)